Problem 1: Package Delivery (Uniform Cost Search)

a) UCS Algorithm to Find Lowest-Cost Path

implement a Uniform Cost Search (UCS) to find the lowest-cost path for a robot that starts at (0, 0) and needs to visit a set of delivery destinations. The terrain has different movement costs, and the robot may move north, south, east, or west

In [ ]:
import heapq
from typing import List, Tuple, Set


terrain_cost = {'R': 1, 'T': 3, 'C': 5, 'E': 0.5}
directions = [(-1, 0), (1, 0), (0, 1), (0, -1)]  # N, S, E, W

def ucs_package_delivery(grid: List[List[str]], start: Tuple[int, int], delivery_points: Set[Tuple[int, int]]):
    rows, cols = len(grid), len(grid[0])
    pq = [(0, start, frozenset(), [start])]
    visited = set()

    while pq:
        cost, (x, y), delivered, path = heapq.heappop(pq)
        state = (x, y, delivered)
        if state in visited:
            continue
        visited.add(state)

        # deliver package if this location is a delivery point
        if (x, y) in delivery_points:
            delivered = frozenset(set(delivered) | {(x, y)})

        # goal condition: all delivery points visited
        if delivered == delivery_points:
            return cost, path

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                terrain = grid[nx][ny]
                move_cost = terrain_cost[terrain]
                heapq.heappush(pq, (cost + move_cost, (nx, ny), delivered, path + [(nx, ny)]))

    return float('inf'), []


In [ ]:
grid = [
    ['R', 'T', 'R', 'E'],
    ['R', 'C', 'T', 'R'],
    ['E', 'R', 'R', 'T'],
    ['R', 'T', 'E', 'R']
]

start = (0, 0)
delivery_points = {(0, 3), (2, 0), (3, 2)}  # example destinations

cost, path = ucs_package_delivery(grid, start, delivery_points)

print("Total Cost:", cost)
print("Path:", path)


Total Cost: 9.5
Path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (2, 2), (1, 2), (0, 2), (0, 3)]


b) Analysis of Scalability

R = number of rows

C = number of columns

D = number of delivery points

Time Complexity:

each state includes a position and the subset of delivery points visited

possible positions = R*C

possible subsets of deliveries = 2^D

so total states = O(R*C*2^D)

each insertion into the priority queue is O(logN), giving total complexity:

O(R*C*2^D*log(R*C*2^D)

Space Complexity:

visited states = O(R*C*2^D)

Scaling Behavior:

efficient on large grids if number of delivery points D is small

becomes exponentially slower as D increases due to the 2^D growth of subsets

 c) One-Way Streets Modification

 to support one-way streets, we need to track which directions are allowed from each cell

 changes needed:

 extend each grid cell to store allowed movement directions:

In [ ]:
# example:
grid_info = {
    (0, 0): {'type': 'R', 'allowed': ['S', 'E']},
    ...
}


In [ ]:
#modify direction lookup to respect one-way rules

direction_map = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}

for d in grid_info[(x, y)]['allowed']:
    dx, dy = direction_map[d]
    ...

keep terrain costs and delivery logic the same

Summary of Effect:

search space reduces due to fewer movement options

correctness maintained by only allowing legal moves

UCS logic stays the same — only the neighbors change

Problem 2: Network Routing (A* Search)

a) Implement A* Search to Find Optimal Route Between Two Servers




In [1]:
import heapq

def calculate_cost(latency, bandwidth, load):
    return latency * (1 + load / 100) / bandwidth

def a_star(graph, start, goal, heuristic):
    """
    graph: dict of nodes and edges with latency, bandwidth, load
        Example: {
            'A': {'B': {'latency': 10, 'bandwidth': 100, 'load': 20}, ...},
            ...
        }
    start: starting node
    goal: goal node
    heuristic: function h(n) estimating cost from n to goal
    """
    frontier = []
    heapq.heappush(frontier, (0, start, [start]))  # (f_score, node, path)
    g_cost = {start: 0}

    while frontier:
        f_score, current, path = heapq.heappop(frontier)

        if current == goal:
            return path, g_cost[current]

        for neighbor, params in graph[current].items():
            edge_cost = calculate_cost(params['latency'], params['bandwidth'], params['load'])
            new_cost = g_cost[current] + edge_cost

            if neighbor not in g_cost or new_cost < g_cost[neighbor]:
                g_cost[neighbor] = new_cost
                h_cost = heuristic(neighbor, goal)
                total_cost = new_cost + h_cost
                heapq.heappush(frontier, (total_cost, neighbor, path + [neighbor]))

    return None, float('inf')


In [5]:
graph = {
    'A': {
        'B': {'latency': 10, 'bandwidth': 100, 'load': 20},
        'C': {'latency': 20, 'bandwidth': 50,  'load': 10},
    },
    'B': {
        'D': {'latency': 30, 'bandwidth': 75,  'load': 50},
    },
    'C': {
        'D': {'latency': 10, 'bandwidth': 50,  'load': 0},
    },
    'D': {}
}

start = 'A'
goal = 'D'


path, total_cost = a_star(graph, start, goal, heuristic)
print("Optimal path:", path)
print("Total cost:", total_cost)


Optimal path: ['A', 'B', 'D']
Total cost: 0.72


 b) Design and Implement an Admissible Heuristic

 to ensure admissibility, the heuristic must never overestimate. A good admissible heuristic for this network routing problem:

In [ ]:
# use the minimum possible latency-to-bandwidth ratio seen across the network as a scaling factor
def build_min_latency_bandwidth(graph):
    min_ratio = float('inf')
    for u in graph:
        for v in graph[u]:
            latency = graph[u][v]['latency']
            bandwidth = graph[u][v]['bandwidth']
            if bandwidth > 0:
                ratio = latency / bandwidth
                if ratio < min_ratio:
                    min_ratio = ratio
    return min_ratio

def heuristic_factory(min_ratio):
    """
    returns a heuristic function h(n, goal) that uses min_ratio.
    assumes each hop is at least one edge with the best possible latency/bandwidth.
    """
    def h(n, goal):
        # admissible estimate: assume 1-hop with best possible ratio
        return min_ratio  # or scaled by estimated hops if graph geometry is known
    return h



c) How does your heuristic handle network congestion?

Network congestion is modeled through load, which affects cost.

To handle it:

During Search: We include load in the true cost calculation (compute_cost) so congested paths are penalized.

In Heuristic:
We avoid using current load in the heuristic to maintain admissibility (load may increase, so ignoring it underestimates cost).

This allows A* to:

Prefer less congested routes.

Adapt routing away from heavily loaded paths.

React if alternate paths become more optimal due to dynamic load changes.



Problem 3: Resource Management (Dynamic Programming)

a) Dynamic Programming Implementation

In [7]:
def max_points(N, M, P, E):
    from functools import lru_cache

    # DP with memoization
    @lru_cache(maxsize=None)
    def dp(i, energy, prev1, prev2):
        if i == N:
            return 0  # no more units left

        max_score = 0

        # option 1: idle
        max_score = max(max_score, dp(i + 1, energy, 'I', prev1))

        # option 2: processing
        if prev1 != 'P' and energy >= E:
            max_score = max(max_score, P + dp(i + 1, energy - E, 'P', prev1))

        # option 3: boosted
        if not (prev1 == 'B' and prev2 == 'B') and energy >= 3 * E:
            max_score = max(max_score, 2 * P + dp(i + 1, energy - 3 * E, 'B', prev1))

        return max_score

    return dp(0, M, 'I', 'I')  # start with first unit, full energy, no prior state


In [8]:
N = 5     # number of units
M = 10    # total energy budget
P = 2     # points for Processing
E = 1     # energy for Processing

print("Maximum Points:", max_points(N, M, P, E))


Maximum Points: 14


b) Proof of Optimality

The solution uses top-down dynamic programming with memoization, which ensures:

Every valid configuration is explored.

No configuration is repeated thanks to caching (@lru_cache).

Transitions respect all constraints:

No more than 2 boosted in a row.

No two processing units without idle in between.

Total energy consumption is always checked.

Therefore, it guarantees an optimal solution by exhaustively checking valid combinations under constraints.

c) Time and Space Complexity

let’s define:

N = number of units

M = energy budget

States per unit: combinations of previous 2 states = 3 * 3 = 9 (I, P, B)

Time Complexity:

Each state is defined by (i, energy, prev1, prev2)

Total number of states = N * M * 3 * 3 = O(9NM) = O(NM)

So the time complexity is O(N * M)

Space Complexity:

Space for memoization table: O(N * M * 3 * 3) = O(NM)

Additional stack space: O(N) for recursion

So the space complexity is also O(N * M)


